In [1]:
# Import Dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
# Create Engine for census data
engine = create_engine("sqlite:///../../data/FPA_FOD_20170508.sqlite")
conn = engine.connect()

In [3]:
# Query All Records in the the Fires Table
fires_data = pd.read_sql("SELECT * FROM Fires", conn)

In [4]:
len(fires_data)

1880465

In [5]:
fires_data.head(10)

,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,0511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,063,Plumas,b'\x00\x01\xad\x10\x00\x00\xe8d\xc2\x92_@^\xc0...
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,061,Placer,b'\x00\x01\xad\x10\x00\x00T\xb6\xeej\xe2\x19^\...
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00\xd0\xa5\xa0W\x13/^\...
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00\x94\xac\xa3\rt\xfa]...
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,003,Alpine,b'\x00\x01\xad\x10\x00\x00@\xe3\xaa.\xb7\xfb]\...
5,6,6,FS-1418849,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.635278,-120.103611,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00\xf0<~\x90\xa1\x06^\...
6,7,7,FS-1418851,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.688333,-120.153333,5.0,USFS,CA,17,017,El Dorado,b'\x00\x01\xad\x10\x00\x00$o\x996\xd0\t^\xc0h\...
7,8,8,FS-1418854,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,40.968056,-122.433889,13.0,STATE OR PRIVATE,CA,None,None,None,b'\x00\x01\xad\x10\x00\x00t)\xe8\xd5\xc4\x9b^\...
8,9,9,FS-1418856,FED,FS-FIRESTAT,FS,USCASHF,Shasta-Trinity National Forest,0514,Shasta-Trinity National Forest,...,B,41.233611,-122.283333,13.0,STATE OR PRIVATE,CA,None,None,None,"b'\x00\x01\xad\x10\x00\x00\xdc\x8d\x1e""""\x92^\..."
9,10,10,FS-1418859,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,0503,Eldorado National Forest,...,A,38.548333,-120.149167,5.0,USFS,CA,5,005,Amador,b'\x00\x01\xad\x10\x00\x00dS\\\xf2\x8b\t^\xc0\...


In [26]:
fires_data_trimmed = fires_data[["OBJECTID","FIRE_YEAR","DISCOVERY_DATE","STATE"
                                    ,"FIRE_SIZE","FIRE_SIZE_CLASS"]]
fires_data_trimmed.head(10)

,OBJECTID,FIRE_YEAR,DISCOVERY_DATE,STATE,FIRE_SIZE,FIRE_SIZE_CLASS
0,1,2005,2453403.5,CA,0.10,A
1,2,2004,2453137.5,CA,0.25,A
2,3,2004,2453156.5,CA,0.10,A
3,4,2004,2453184.5,CA,0.10,A
4,5,2004,2453184.5,CA,0.10,A
5,6,2004,2453186.5,CA,0.10,A
6,7,2004,2453187.5,CA,0.10,A
7,8,2005,2453437.5,CA,0.80,B
8,9,2005,2453444.5,CA,1.00,B
9,10,2004,2453187.5,CA,0.10,A


In [27]:
epoch = pd.to_datetime(0, unit='s').to_julian_date()
fires_data_trimmed["DISCOVERY_DATE"] = pd.to_datetime(fires_data_trimmed["DISCOVERY_DATE"] - epoch, unit='D')
fires_data_trimmed_converted = fires_data_trimmed

<ipython-input-27-71ef12ffd381>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fires_data_trimmed["DISCOVERY_DATE"] = pd.to_datetime(fires_data_trimmed["DISCOVERY_DATE"] - epoch, unit='D')


In [28]:
fires_data_yearly_df = pd.DataFrame(fires_data_trimmed_converted)
print(len(fires_data_yearly_df))
fires_data_yearly_df.head(5) 


1880465


,OBJECTID,FIRE_YEAR,DISCOVERY_DATE,STATE,FIRE_SIZE,FIRE_SIZE_CLASS
0,1,2005,2005-02-02,CA,0.10,A
1,2,2004,2004-05-12,CA,0.25,A
2,3,2004,2004-05-31,CA,0.10,A
3,4,2004,2004-06-28,CA,0.10,A
4,5,2004,2004-06-28,CA,0.10,A


In [29]:
# grouping by year
year_group = fires_data_yearly_df.groupby(['FIRE_YEAR'])
year_group

In [30]:
# grouping by year & state
year_state_group = fires_data_yearly_df.groupby(['FIRE_YEAR','STATE'])
year_state_group

In [38]:
# total fires by year
total_fires_year = year_group['OBJECTID'].count()
total_fires_year_df = pd.DataFrame(total_fires_year).rename(columns={"OBJECTID": "COUNT_FIRES"})
total_fires_year_df

,COUNT_FIRES
FIRE_YEAR,
1992,67975
1993,61989
1994,75955
1995,71472
1996,75574
1997,61450
1998,68370
1999,89363
2000,96416


In [39]:
# total fires by year by state
total_fires_year_state = year_state_group['OBJECTID'].count()
total_fires_year_state_df = pd.DataFrame(total_fires_year_state).rename(columns={"OBJECTID": "COUNT_FIRES"})
total_fires_year_state_df

COUNT_FIRES
FIRE_YEAR STATE             
1992      AK             481
          AL              93
          AR             128
          AZ            3494
          CA           10833
...                      ...
2015      VT              87
          WA            1741
          WI             965
          WV             685
          WY             632

[1214 rows x 1 columns]

In [34]:
# total fire size by year
total_fire_size_year = year_group['FIRE_SIZE'].sum()
total_fire_size_year_df = pd.DataFrame(total_fire_size_year)
total_fire_size_year_df

,FIRE_SIZE
FIRE_YEAR,
1992,2.199958e+06
1993,2.191696e+06
1994,4.116676e+06
1995,2.049585e+06
1996,6.005156e+06
1997,3.214635e+06
1998,2.011102e+06
1999,6.080200e+06
2000,7.639499e+06


In [40]:
# total fire size by year by state
total_fire_size_year_state = year_state_group['FIRE_SIZE'].sum()
total_fire_size_year_state_df = pd.DataFrame(total_fire_size_year_state)
total_fire_size_year_state_df

FIRE_SIZE
FIRE_YEAR STATE             
1992      AK      142717.400
          AL        1253.200
          AR        1478.500
          AZ       42593.650
          CA      296439.800
...                      ...
2015      VT         346.429
          WA     1161646.380
          WI        2721.440
          WV       17251.370
          WY       36821.225

[1214 rows x 1 columns]

In [42]:
# merging count of fires and sum of size dataframes
fire_size_count_year_state = total_fire_size_year_state_df.merge(total_fires_year_state_df
                                    ,left_index=True,right_index=True)
fire_size_count_year_state_df = pd.DataFrame(fire_size_count_year_state)
fire_size_count_year_state_df

FIRE_SIZE  COUNT_FIRES
FIRE_YEAR STATE                          
1992      AK      142717.400          481
          AL        1253.200           93
          AR        1478.500          128
          AZ       42593.650         3494
          CA      296439.800        10833
...                      ...          ...
2015      VT         346.429           87
          WA     1161646.380         1741
          WI        2721.440          965
          WV       17251.370          685
          WY       36821.225          632

[1214 rows x 2 columns]

In [45]:
fire_size_count_year_state_df['AVG_FIRE_SIZE'] = fire_size_count_year_state_df['FIRE_SIZE'] / fire_size_count_year_state_df['COUNT_FIRES']
fire_size_count_year_state_df

FIRE_SIZE  COUNT_FIRES  AVG_FIRE_SIZE
FIRE_YEAR STATE                                         
1992      AK      142717.400          481     296.709771
          AL        1253.200           93      13.475269
          AR        1478.500          128      11.550781
          AZ       42593.650         3494      12.190512
          CA      296439.800        10833      27.364516
...                      ...          ...            ...
2015      VT         346.429           87       3.981943
          WA     1161646.380         1741     667.229397
          WI        2721.440          965       2.820145
          WV       17251.370          685      25.184482
          WY       36821.225          632      58.261432

[1214 rows x 3 columns]

In [47]:
# exporting total fires by year by state
output_path_fires_year_state = "output/total_fires_yearly_by_state.csv"
fire_size_count_year_state_df.to_csv(output_path_fires_year_state,index=True)